## scipy.optimize中的非线性规划算法

单变量无约束优化：optimize.minimize_scaler()
- 'brute'
- 'bounded': 可加范围
<!-- 举任意例子 -->


多变量局部优化：optimize.minimize()
- 有jac, hess可以写上去
    - 'Newton-CG'
    - 'BFGS'
    - 'L-BFGS-B'
- constraint用list，规范写
    - 'SLSQP'
- 可以加上bounds
    - 'Nelder-Mead': 不依赖导数
<!-- 举rosenbrock例子 -->


多变量全局优化：
- optimize.shgo: 单纯同调全局优化
- optimize.dual_annealing: 模拟退火，避免局部最优解
- optimize.differential_evolution: 差分进化
- optimize.basinhopping: 盆地跳跃.
<!-- 举eggholder例子 -->


https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.shgo.html?highlight=shgo#scipy.optimize.shgo

https://docs.scipy.org/doc/scipy/tutorial/optimize.html#global-optimization


## cvxpy中的优化算法

凸优化问题
最大最小问题
混合整数规划问题

prob.status, prob.value

curvature、sign计算规则

DCP规则:记忆为保持单纯形

*,@,cp.multiply()

**==cp.power() 针对元素

cp.sum()只针对矩阵，lambda相关函数需要S

cp.sum(axis=0,1)中的keepdims=False默认squeeze

cp.conv(x)卷积，cp.vec(X)按列展成向量，cp.reshape(X, (m,n), order='C','F')

DGP: prob.solve(gp=True) 把乘法看成log-long affine

DQCP: quasiconvex定义域和sublevel sets为凸集; 规则加入quasiconvex<=constant, quasiconcave>=constant; prob.solve(qcp=True); 取整，标量乘除法；prob.solve(qcp=True, low=, high=); 有时候会error,可以考虑更换求解器

求解器cp.installed_solvers()    ['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

半正定 x=cp.Variable((3, 3), PSD=True)

01规划 boolean=True, 混合整数规划 integer=True



关于求解器

- 整数规划

The preferred open source mixed-integer solvers in CVXPY are GLPK_MI, CBC and SCIP. 

SCIP supports nonlinear models, but **GLPK_MI** and CBC do not.

CVXPY comes with ECOS_BB – an open source mixed-integer nonlinear solver – by default. However ECOS_BB will not be called automatically; you must explicitly call prob.solve(solver='ECOS_BB') if you want to use it (changed in CVXPY 1.1.6). This policy stems from the fact that there are recurring correctness issues with ECOS_BB. If you rely on this solver for some application then you need to be aware of the increased risks that come with using it. If you need to use an open-source mixed-integer nonlinear solver from CVXPY, then we recommend you install SCIP.

- 参数

solve(solver=None, verbose=False, gp=False, qcp=False, requries_grad=False, enforce_dpp=False, **kwargs)

Gradients are only supported for DCP and DGP problems, not quasiconvex problems. When computing gradients (i.e., when this argument is True), the problem must satisfy the DPP rules.

- 默认求解器

By default CVXPY calls the solver most specialized to the problem type. For example, ECOS is called for SOCPs. SCS can handle all problems (except mixed-integer programs). If the problem is a QP, CVXPY will use OSQP.


![](https://cdn.jsdelivr.net/gh/acryan54/images@main/pic/202207230228877.png)
LP线性规划 QP二次规划 SOCP二阶锥规划 SDP半正定规划 EXP POW MIP混合整数规划

In [59]:
import numpy as np
import cvxpy as cp
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

In [57]:
import numpy as np
import cvxpy as cp

x = cp.Variable()
a = cp.Parameter(nonneg=True)
# x**3
# cp.sqrt(1 + x**2).curvature
# cp.norm2(cp.hstack([1, x])).curvature
# cp.sqrt(x).curvature
# x.is_concave()


'QUASICONVEX'

In [37]:
x = cp.Variable()
y = cp.Variable()

# DCP problems.
prob1 = cp.Problem(cp.Minimize(cp.square(x - y)),
                    [x + y >= 0])
prob2 = cp.Problem(cp.Maximize(cp.sqrt(x - y)),
                [2*x - 3 == y,
                 cp.square(x) <= 2])

print("prob1 is DCP:", prob1.is_dcp())
print("prob2 is DCP:", prob2.is_dcp())

# Non-DCP problems.

# A non-DCP objective.
obj = cp.Maximize(cp.square(x))
prob3 = cp.Problem(obj)

print("prob3 is DCP:", prob3.is_dcp())
print("Maximize(square(x)) is DCP:", obj.is_dcp())

# A non-DCP constraint.
prob4 = cp.Problem(cp.Minimize(cp.square(x)),
                    [cp.sqrt(x) <= 2])

print("prob4 is DCP:", prob4.is_dcp())
print("sqrt(x) <= 2 is DCP:", (cp.sqrt(x) <= 2).is_dcp())

prob1 is DCP: True
prob2 is DCP: True
prob3 is DCP: False
Maximize(square(x)) is DCP: False
prob4 is DCP: False
sqrt(x) <= 2 is DCP: False


In [47]:
x = cp.Variable(5)
print("0 dimensional", x[0].shape)
print("1 dimensional", x[0:1].shape)

0 dimensional ()
1 dimensional (1,)


[1, 2, 3, 4, 5]

In [50]:
X = cp.Variable((5, 4))
col_sums = cp.sum(X, axis=0, keepdims=True) # Has size (1, 4)
col_sums = cp.sum(X, axis=0) # Has size (4,)
row_sums = cp.sum(X, axis=1) # Has size (5,)

In [55]:
import cvxpy as cp

# DGP requires Variables to be declared positive via `pos=True`.
x = cp.Variable(pos=True)
y = cp.Variable(pos=True)
z = cp.Variable(pos=True)

objective_fn = x * y * z
constraints = [
  4 * x * y * z + 2 * x * z <= 10, x <= 2*y, y <= 2*x, z >= 1]
problem = cp.Problem(cp.Maximize(objective_fn), constraints)
problem.solve()
print("Optimal value: ", problem.value)
print("x: ", x.value)
print("y: ", y.value)
print("z: ", z.value)

DCPError: Problem does not follow DCP rules. Specifically:
The objective is not DCP. Its following subexpressions are not:
var385 @ var386
However, the problem does follow DGP rules. Consider calling solve() with `gp=True`.

In [58]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']